# Data processing with `pandas` to draw a Sankey diagram

Most of the material in this notebook was taken from Lukas Gast contributions to coding tutorials for the Resource Efficiency Collective in Cambridge. Lukas was a Research Assistant in the group while and after completing his PhD in Engineering. 

## Figuring out a large table

The UK publishes very detailed data on waste collection and use. These data can help us map out a significant part of the material flowing in the UK and identify opportunities where the system could be improved. The UK's Department for Environment Food & Rural Affairs (DEFRA) produces reports from these data, such as [this](https://www.gov.uk/government/statistics/local-authority-collected-waste-management-annual-results), where results are reported in a way that is too aggregated to be useful.

The raw data on UK waste is a perfect example to illustrate how `pandas` can be used to process data. In this example we will use `pandas` to "undestand" a large data set and process these data into a state that can be used to plot diagrams.

Let's start by importing the raw data,

In [1]:
import pandas as pd
waste_data = pd.read_csv('data/Q100_Waste_collection_data_England_2019-20.csv',
                         encoding = 'iso-8859-1', 
                         header = 1)

waste_data

,WasteProcessorId,WasteStreamId,WasteProcessorOutputId,SenderWasteProcessorOutputId,Authority,AuthorityId,Period,PeriodId,WasteStreamTypeId,WasteStreamType,...,TonnesFromCommercialSources,TonnesFromIndustrialSources,TonnesFromNonHHSources,TonnesFromWfHSources,TonnesFromWnfHSources,UsageId,Usage,QuarterlyComments,MonthlyComments,MaterialGroup
0,3858654,3857229,0,0,Luton Borough Council,1,Apr 19 - Jun 19,307,1,Source segregated recyclate,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,Glass
1,3858654,3857229,14131667,0,Luton Borough Council,1,Apr 19 - Jun 19,307,1,Source segregated recyclate,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,Glass
2,3858655,3857223,0,0,Luton Borough Council,1,Apr 19 - Jun 19,307,1,Source segregated recyclate,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,Tyres
3,3858655,3857223,14131668,0,Luton Borough Council,1,Apr 19 - Jun 19,307,1,Source segregated recyclate,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,Tyres
4,3858656,3857231,0,0,Luton Borough Council,1,Apr 19 - Jun 19,307,1,Source segregated recyclate,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,Textiles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231533,4023550,4022113,0,0,Corby and Kettering Shared Service,562,Jan 20 - Mar 20,319,5,Residual waste,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
231534,4023550,4022113,14988130,0,Corby and Kettering Shared Service,562,Jan 20 - Mar 20,319,5,Residual waste,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,Organic
231535,4023550,4022113,14988175,14988130,Corby and Kettering Shared Service,562,Jan 20 - Mar 20,319,5,Residual waste,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,Organic
231536,4023550,4022113,14988131,0,Corby and Kettering Shared Service,562,Jan 20 - Mar 20,319,5,Residual waste,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,Other Materials


The code above creates a `DataFrame`, called `waste_data`, by importing all the data within `Q100_Waste_collection_data_England_2019-20.csv`. A `DataFrame` is a data structure that can store various data types. The [python documentation](https://docs.python.org/3/library/stdtypes.html) explains the main data types availabe in python.
The parameter `header = 1` means that the headings for the columns of the dataframe are the second line of the raw data file.

The data in this dataset are not arranged intuitively and are difficult to compreend by most humans.

We are interested in how material flows within the UK so let's undertand which materials are covered by `waste_data`,

In [2]:
material_group = waste_data['MaterialGroup'].drop_duplicates()
material_group

0                  Glass
2                  Tyres
4               Textiles
6                  Metal
8                    Oil
10                  Wood
14       Other Materials
18          Plasterboard
20             Batteries
24                  WEEE
28          Paper & Card
30                Rubble
32                  Soil
34                   NaN
46               Plastic
50               Organic
319            Furniture
458                  IBA
595            Composite
2369               Bulky
5933               Paint
55715         Co-mingled
Name: MaterialGroup, dtype: object

The cell above applied the `drop_duplicates()` method of `pandas` to produce a list of all the unique values in the `MaterialGroup` column of `waste_data`. This a well organised list and there is not a lot of overlap between the items (which is promising).

The `Material` column provides a more detailed description of the type of objects described by the data. Let's produce a list of all the objects covered by the data,

In [3]:
object_types = waste_data.loc[waste_data['MaterialGroup'].notnull(), ['Material']].drop_duplicates().values
object_types

array([['Mixed glass'],
       ['Car tyres'],
       ['Carpets'],
       ['Gas bottles'],
       ['Vegetable Oil'],
       ['Wood'],
       ['Textiles & footwear'],
       ['Bric-a-brac'],
       ['Mineral Oil'],
       ['Plasterboard'],
       ['Automotive batteries'],
       ['Steel cans'],
       ['WEEE - Large Domestic App'],
       ['Aluminium cans'],
       ['Card'],
       ['Rubble'],
       ['Soil'],
       ['Paper'],
       ['Mixed Plastic Bottles'],
       ['Waste food only'],
       ['Green garden waste only'],
       ['Mixed Plastics'],
       ['Post consumer, non automotive batteries'],
       ['WEEE - Flourescent tubes and other light bulbs'],
       ['Other Scrap metal'],
       ['Mixed tyres'],
       ['WEEE - Fridges & Freezers'],
       ['WEEE - TVs & Monitors'],
       ['Green glass'],
       ['Brown glass'],
       ['WEEE - Small Domestic App'],
       ['Other materials'],
       ['Furniture'],
       ['Video tapes, DVDs and CDs'],
       ['Bicycles'],
       ['Meta

In the cell, above we are excluding all the rows that do not have a `MaterialGrouping`. We are assuming that if the data point does not have a `MaterialGrouping` it does not represent a material. This is accomplished by chaining the `loc()` and `notnull()` functions. The `loc()` function selects all the rows were `waste_data['MaterialGroup'].notnull() == True`. This produces a list of 63 object types.

These object types are not mutually exclusive as, for example, `"WEEE - Fridges & Freezers"` might contain `"PS [6]"`. This is relevant if we are trying to map the flow of materials within a production system.

We can understand this dataset better by plotting a simple sankey diagram, where the "source" are the local authorities and the "target" is the object type. We first need to use pandas to aggregate the data.

In [4]:
flows = waste_data.loc[
    ~waste_data['FacilityType'].isin(['Final Destination']) & 
    waste_data['TotalTonnes'].notnull() &
    waste_data['Material'].notnull(),
    [
        'Authority',
        'MaterialGroup',
        'WasteStreamType', 
        'FacilityType', 
        'FacilityName', 
        'OutputProcessType', 
        'Material', 
        'TotalTonnes'
    ]
].groupby([
        waste_data['Authority'],
        waste_data['MaterialGroup'],
        waste_data['WasteStreamType'], 
        waste_data['FacilityType'], 
        waste_data['FacilityName'], 
        waste_data['OutputProcessType'], 
        waste_data['Material'],         
    ]).agg(sum).reset_index().rename(columns = {
    'Authority': 'source',
    'MaterialGroup': 'target',
    'TotalTonnes': 'value',
})

flows.to_csv('data/flows.csv')
flows

,source,target,WasteStreamType,FacilityType,FacilityName,OutputProcessType,Material,value
0,Adur District Council,Glass,Comingled recyclate,Reprocessor - recycling (qu19),Chambers Waste Management Plc,Dry recyclate,Mixed glass,432.72
1,Adur District Council,Glass,Comingled recyclate,Reprocessor - recycling (qu19),Crossways Recycling Limited,Dry recyclate,Mixed glass,0.05
2,Adur District Council,Glass,Comingled recyclate,Reprocessor - recycling (qu19),Glass Recycling ( U K) Ltd,Dry recyclate,Mixed glass,6.32
3,Adur District Council,Glass,Comingled recyclate,Reprocessor - recycling (qu19),Other/Exempt,Dry recyclate,Mixed glass,1067.68
4,Adur District Council,Glass,Comingled recyclate,Reprocessor - recycling (qu19),R Collard Limited,Dry recyclate,Mixed glass,191.21
...,...,...,...,...,...,...,...,...
11934,York City Council,Plastic,Residual waste,Reprocessor - recycling (qu19),Biffa Waste Services Limited,Dry recyclate,Mixed Plastics,3.17
11935,York City Council,Plastic,Residual waste,Reprocessor - recycling (qu19),Clean Tech Europe Limited,Dry recyclate,Mixed Plastics,119.56
11936,York City Council,Plastic,Residual waste,Reprocessor - recycling (qu19),Other/Exempt,Dry recyclate,Mixed Plastic Bottles,57.55
11937,York City Council,Plastic,Residual waste,Reprocessor - recycling (qu19),Other/Exempt,Dry recyclate,Mixed Plastics,315.44


The `flows` dataframe is in a format that can be used by `floweaver` to produce a sankey diagram. We achieved this in two stages:
  1. Selected the rows that have mass data and ignored the rows that had `Final destination` in the `FacilityType`. We need to do this to avoid double counting.
  2. Added up all the values for every unique combination of all the columns that are not `TotalTonnes` and renamed some columns to names that `floweaver` understands.
  
Note that we went from a table with ~ 200k rows to around 11k.

The above is a good example on how simply we can process tables using `pandas` thanks to the ability of chaining methods.

See how these data are used to draw a sankey diagram in the `sankey_diagrams.ipynb` file.